### Libraries

In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import datetime
import json
import time
import os
import pandas as pd

In [ ]:

date_list = [datetime.datetime.strptime('20200123  17:00:00', '%Y%m%d %H:%M:%S')
  -datetime.timedelta(days=i+1) for i in range(0, (365*5))]
len(date_list)

In [ ]:
date_list = [i.strftime("%Y%m%d %H:%M:%S") for i in date_list if i.weekday() < 5]
len(date_list)

In [ ]:
class TestApp(EWrapper, EClient):

    def __init__(self):
        EClient.__init__(self,self)
        self.list_of_dict= {}
    def error(self, reqId, errorCode, errorString):
        time.sleep(.5)
        print("Error:", reqId, " ", errorCode, " ", errorString)
            
    def historicalData(self, reqId:int, bar):
        print(bar.date)
        self.list_of_dict[bar.date] = {'Open':bar.open, 
                                       'High':bar.high, 'Low':bar.low, 'Close':bar.close, 'Volume':bar.volume}
        time.sleep(.5)
        
        
    def historicalDataEnd(self, reqId: int, start: str, end: str):
        super().historicalDataEnd(reqId, start, end)
        print("done")
        self.done = True
        self.disconnect()    

### Entering Virtus LifeSci Biotech ETF Portifolio

In [ ]:
df = pd.read_excel('positions_bbc.xls', index_col=0, skiprows=1)

#### Creating a dictionary for each stock as the key and list of dates as the value

In [ ]:
stock_date = {i:{'dates':date_list, 'symbol': i, 'primaryExchange':df.Primary.iloc[index]} for index,i in enumerate(df.Ticker.values)}

#### Creating a dictionary to collect stock info.

In [ ]:
stock_dict = {}

In [ ]:
for i in df.Ticker.values:
    app = TestApp()
    #app.connect("127.0.0.1", 7496, 0)
    contract = Contract()
    contract.symbol = stock_date[i]['symbol']
    contract.primaryExchange = stock_date[i]['primaryExchange']
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    for j,date in enumerate(stock_date[i]['dates']):
        app.connect("127.0.0.1", 7496, 0)
        app.reqHistoricalData(j, contract, date, '1 D', '1 min', 'TRADES', 1, 1, False, [])
        app.run()
        time.sleep(10)
        stock_dict[i] = app.list_of_dict
        print(j)
    
    with open('result.json', 'w') as fp:
            json.dump(stock_dict, fp)
    
    
    